In [3]:
# Install liblinear: sudo apt-get install python-liblinear
# from liblinearutil import *
import numpy as np

In [4]:
def read_problem(file_name):
    y = []
    x = []
    feature_len = 22
    for line in open(file_name):
        tmp = line.split(' ',1)
        y = y + [int(tmp[0])]
        vec = np.zeros(feature_len)
        for each in tmp[1].split():
            index,val = each.split(':')
            vec[int(index)-1] = float(val)
        x.append(vec)
    y = np.array(y)
    x = np.array(x)
    return y,x

In [3]:
y,x = read_problem('./data/ijcnn1.tr')

# Gradient ascent step for dual variables

In [6]:
def get_tp_tn(x,y,w):
    h = x.dot(w)
    ind = np.where(h>0)
    y1 = -1*np.ones_like(y)
    y1[ind] = 1
    tp = sum(np.multiply(1+y,1+y1)/4)
    tn = sum(np.multiply(1-y,1-y1)/4)
    return tp,tn

def gradient_ascent(x,y,w,a,b,alpha=0.01):
    n = x.shape[0]
    n_pos = len([i for i in range(n) if y[i]==1])
    n_neg = n-n_pos
    ind_pos = [i for i in range(n) if y[i]>0]
    ind_neg = [i for i in range(n) if y[i]<0]
    tpr,tnr = get_tp_tn(x,y,w)
    tpr /= float(n_pos)
    tnr /= float(n_neg)
    print('TPR =',tpr,'TNR =',tnr)
    for i in range(n):
        g = 0
        if i in ind_pos:
            try:
                tmp = 1 - (2*n_pos)/(a*n)*(x[i].dot(w))
            except:
                tmp = 0
            if tmp > 0:
                g += 1/n_pos
            g = g-1/n_pos+tpr
            a = a + alpha*g
        else:
            try:
                tmp = 1 + (2*n_neg)/(b*n)*(x[i].dot(w))
            except:
                tmp = 0
            if tmp > 0:
                g += 1/n_neg
            g = g-1/n_neg+tnr
            b = b + alpha*g
        # Projecttion to a,b \in R+
        if a<0:
            a=0
        if b<0:
            b=0
    return a,b

In [5]:
w = np.random.uniform(low=-1,high=1,size=(22,))
a,b = 0.1,0.1
a,b=gradient_ascent(x,y,w,a,b)
print(a,b)

TPR = 0.616983894583 TNR = 0.4372645243
21.1699824305 138.209978471


## Liblinear for primal variable w

In [1]:
# customized liblinear
import sys
sys.path.append('/home/debojyoti/opt/liblinear-2.1')
from ppython import liblinear
from ppython.liblinear import *
from ppython.liblinearutil import *

In [ ]:
# m = train(y,x,'-w1 4 -c 5')

In [11]:
y,x = svm_read_problem('./data/ijcnn1.tr')
prob = problem(y,x)
# Compute parameters
n = len(y)
n_pos = len([i for i in range(n) if y[i]==1])
n_neg = n-n_pos
c1 = a/n_pos
c2 = b/n_neg
param = parameter('-w-1 {} -c {}'.format(c2,c1))
model = train(prob,param)
# Get model parameters
model.get_decfun()

[-1] [0.0043758106212060345]


([0.49014455906442256,
  0.497657498955292,
  0.45977295420147596,
  0.5325482485480034,
  0.5375778891294267,
  0.4956513391033062,
  0.46963508373998464,
  0.5086245899909636,
  0.517961176022266,
  0.5227355068645302,
  -0.13811825454967516,
  0.8911971814141921,
  0.006245671682481706,
  -0.010075546481659717,
  -0.02694263740456741,
  -0.05830889011411127,
  -0.41751717582265785,
  -0.5879909308554518,
  -0.32555054243294634,
  0.10953657275671233,
  0.06368018685880038,
  0.04195977414708689],
 0.0)

In [13]:
# init section
w = np.random.uniform(low=-1,high=1,size=(22,))
a,b = 0.1,0.1
y_lst,x_lst = read_problem('./data/ijcnn1.tr')
y,x = svm_read_problem('./data/ijcnn1.tr')
prob = problem(y,x)
n = len(y)
n_pos = len([i for i in range(n) if y[i]==1])
n_neg = n-n_pos
c = 5
# Iterative section: Gradient Ascent & Liblinear
for i in range(10):
    a,b = gradient_ascent(x_lst,y_lst,w,a,b) #Gradient ascent
    c1 = a/n_pos
    c2 = b/n_neg
    param = parameter('-w1 {} -w-1 {} -c {}'.format(c1,c2,c))
    model = train(prob,param)
    # Get model parameters
    w = model.get_decfun()[0]

TPR = 0.592093704246 TNR = 0.466835523191
TPR = 0.23074670571 TNR = 0.987589045433
TPR = 0.0992679355783 TNR = 0.996992243153
TPR = 0.0576866764275 TNR = 0.998955200253
TPR = 0.0351390922401 TNR = 0.999461769827
TPR = 0.0207906295754 TNR = 0.999683394016
TPR = 0.0128843338214 TNR = 0.999873357606
TPR = 0.00761346998536 TNR = 0.999968339402
TPR = 0.00468521229868 TNR = 1.0
TPR = 0.00204978038067 TNR = 1.0


In [ ]:
# def gradient_ascent_batch(x,y,w,alpha=0.1,ep=0.0001,maxiter=100):
#     n = x.shape[0]
#     n_pos = len([i for i in range(n) if y[i]==1])
#     n_neg = n-n_pos
#     a,b = 0.1,0.1
#     ind_pos = [i for i in range(n) if y[i]>0]
#     ind_neg = [i for i in range(n) if y[i]<0]
#     x_pos = x[ind_pos]
#     x_neg = x[ind_neg]
#     tpr,tnr = get_tp_tn(x,y,w)
#     tpr /= n_pos
#     tnr /= n_neg
#     print 'TPR =',tpr,'TNR =',tnr
#     for i in range(maxiter):
#         # Positive instances
#         tmp = 1 - (2*n_pos)/(a*n)*x_pos.dot(w)
#         j = np.where(tmp>0)
#         tmp = np.zeros_like(tmp)
#         tmp[j]=1/n_pos
#         a += alpha * (sum(tmp)-1+tpr)
#         # Negative instances
#         tmp = 1 + (2*n_neg)/(b*n)*x_neg.dot(w)
#         j = np.where(tmp>0)
#         tmp = np.zeros_like(tmp)
#         tmp[j]=1/n_neg
#         b += alpha * (sum(tmp)-1+tnr)
#         print a,b